# Adding features for days off, days of the week and vacations

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
whole_df = pd.read_csv("/Users/cha/Desktop/Code/eCSpress/data/clean_baseline.csv", sep=";")
whole_df['Date et heure de comptage'] = pd.to_datetime(whole_df['Date et heure de comptage'], format='%Y-%m-%d %H:%M:%S')

print("len whole_df", len(whole_df))
print(whole_df.dtypes)
whole_df.head(3)

len whole_df 27548
Libelle                              object
Date et heure de comptage    datetime64[ns]
Débit horaire                       float64
Taux d'occupation                   float64
Etat trafic                           int64
Etat arc_Barré                        int64
Etat arc_Invalide                     int64
dtype: object


,Libelle,Date et heure de comptage,Débit horaire,Taux d'occupation,Etat trafic,Etat arc_Barré,Etat arc_Invalide
0,AV_Champs_Elysees,2022-01-01 11:00:00,524.0,8.85667,1,0,1
1,AV_Champs_Elysees,2022-01-01 07:00:00,491.0,7.71611,1,0,1
2,AV_Champs_Elysees,2022-01-01 19:00:00,997.0,46.28889,3,0,1


catégorie vacances:
- Départ weekend/vacances
- vacances, Été ?
- Jour de la semaine
- saison
- Moment de la journée

### Jours fériés and vacation

In [10]:
def get_date(row):
    return row.date()

feries = pd.read_csv("/Users/cha/Desktop/Code/eCSpress/data/jours_feries_metropole.csv")
feries = feries[feries['annee'].isin([2021, 2022])]
feries['date'] = pd.to_datetime(feries['date']).apply(get_date)
# series_feries = feries['date']
feries = feries.reset_index()
feries = feries["date"]
feries

0     2021-01-01
1     2021-04-05
2     2021-05-01
3     2021-05-08
4     2021-05-13
5     2021-05-24
6     2021-07-14
7     2021-08-15
8     2021-11-01
9     2021-11-11
10    2021-12-25
11    2022-01-01
12    2022-04-18
13    2022-05-01
14    2022-05-08
15    2022-05-26
16    2022-06-06
17    2022-07-14
18    2022-08-15
19    2022-11-01
20    2022-11-11
21    2022-12-25
Name: date, dtype: object

In [11]:
vacation = pd.read_csv("/Users/cha/Desktop/Code/eCSpress/data/fr-en-calendrier-scolaire.csv", sep=";")
vacation["start_date"] = pd.to_datetime(vacation["start_date"], format='%Y-%m-%d').apply(get_date)
vacation["end_date"] = pd.to_datetime(vacation["end_date"], format='%Y-%m-%d').apply(get_date)
vacation = vacation[(vacation["zones"]=="Zone C") & (vacation["annee_scolaire"].isin(["2021-2022", "2022-2023"])) & (vacation["location"]=="Paris")]
vacation = vacation.reset_index()
vacation

,index,description,population,start_date,end_date,location,zones,annee_scolaire
0,175,Vacances de la Toussaint,-,2021-10-23,2021-11-08,Paris,Zone C,2021-2022
1,183,Vacances d'Hiver,-,2022-02-19,2022-03-07,Paris,Zone C,2021-2022
2,244,Pont de l'Ascension,-,2023-05-18,2023-05-22,Paris,Zone C,2022-2023
3,449,Vacances de Noël,-,2021-12-18,2022-01-03,Paris,Zone C,2021-2022
4,456,Vacances de Printemps,-,2022-04-23,2022-05-09,Paris,Zone C,2021-2022
5,479,Vacances d'Été,Élèves,2022-07-07,2022-09-01,Paris,Zone C,2021-2022
6,484,Vacances de la Toussaint,-,2022-10-22,2022-11-07,Paris,Zone C,2022-2023
7,499,Vacances de Printemps,-,2023-04-22,2023-05-09,Paris,Zone C,2022-2023
8,767,Vacances de Noël,-,2022-12-17,2023-01-03,Paris,Zone C,2022-2023
9,1077,Début des Vacances d'Été,-,2023-07-08,2023-07-08,Paris,Zone C,2022-2023


### Jour de la semaine

In [12]:
whole_df['Date'] = pd.to_datetime(whole_df["Date et heure de comptage"]).dt.date
whole_df['Jour de la semaine'] = pd.to_datetime(whole_df["Date et heure de comptage"]).dt.dayofweek
whole_df['Jour férié'] = whole_df['Date'].isin(feries)
print(whole_df['Jour férié'].value_counts())
whole_df['Jour férié'] = whole_df['Jour férié'].replace(True, 1).replace(False, 0)

whole_df.head()

False    26624
True       924
Name: Jour férié, dtype: int64


,Libelle,Date et heure de comptage,Débit horaire,Taux d'occupation,Etat trafic,Etat arc_Barré,Etat arc_Invalide,Date,Jour de la semaine,Jour férié
0,AV_Champs_Elysees,2022-01-01 11:00:00,524.0,8.85667,1,0,1,2022-01-01,5,1
1,AV_Champs_Elysees,2022-01-01 07:00:00,491.0,7.71611,1,0,1,2022-01-01,5,1
2,AV_Champs_Elysees,2022-01-01 19:00:00,997.0,46.28889,3,0,1,2022-01-01,5,1
3,AV_Champs_Elysees,2022-01-01 18:00:00,1096.0,46.60222,3,0,1,2022-01-01,5,1
4,AV_Champs_Elysees,2022-01-01 09:00:00,258.0,3.75445,1,0,1,2022-01-01,5,1


### Encoding ordinal pour les jours de la semaine

In [13]:
whole_df = pd.concat([
    whole_df,
    pd.get_dummies(whole_df['Jour de la semaine'],
                   prefix='Jour de la semaine',
                   drop_first=False)
], axis=1).drop(columns=['Jour de la semaine'])

whole_df.head()

,Libelle,Date et heure de comptage,Débit horaire,Taux d'occupation,Etat trafic,Etat arc_Barré,Etat arc_Invalide,Date,Jour férié,Jour de la semaine_0,Jour de la semaine_1,Jour de la semaine_2,Jour de la semaine_3,Jour de la semaine_4,Jour de la semaine_5,Jour de la semaine_6
0,AV_Champs_Elysees,2022-01-01 11:00:00,524.0,8.85667,1,0,1,2022-01-01,1,0,0,0,0,0,1,0
1,AV_Champs_Elysees,2022-01-01 07:00:00,491.0,7.71611,1,0,1,2022-01-01,1,0,0,0,0,0,1,0
2,AV_Champs_Elysees,2022-01-01 19:00:00,997.0,46.28889,3,0,1,2022-01-01,1,0,0,0,0,0,1,0
3,AV_Champs_Elysees,2022-01-01 18:00:00,1096.0,46.60222,3,0,1,2022-01-01,1,0,0,0,0,0,1,0
4,AV_Champs_Elysees,2022-01-01 09:00:00,258.0,3.75445,1,0,1,2022-01-01,1,0,0,0,0,0,1,0


In [14]:
whole_df.to_csv("/Users/cha/Desktop/Code/eCSpress/data/baseline_with_vacations.csv", index=False)